<a href="https://colab.research.google.com/github/lonexreb/CiPE/blob/CiPE_working/CiPE_Test3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Om Maa

In [10]:
!pip install langchain predictionguard lancedb html2text sentence-transformers PyPDF2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.9 MB/s eta 0:00:00


In [3]:
import os
import urllib.request

import html2text
import predictionguard as pg
from langchain import PromptTemplate, FewShotPromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from sentence_transformers import SentenceTransformer
import numpy as np
import lancedb
from lancedb.embeddings import with_embeddings
import pandas as pd


os.environ['PREDICTIONGUARD_TOKEN'] = "q1VuOjnffJ3NO2oFN8Q9m8vghYc84ld13jaqdF7E"


In [24]:
# # Chaining

# template = """### Instruction:
# Decide if the following input message is an informational question, a general chat message, or a request for code generation.
# If the message is an informational question, answer it based on the informational context provided below.
# If the message is a general chat message, respond in a kind and friendly manner based on the coversation context provided below.
# If the message is a request for code generation, respond with a code snippet.

# ### Input:
# Message: {query}

# Informational Context: The Greater Los Angeles and San Francisco Bay areas in California are the nation's second and fifth-most populous urban regions, respectively. Greater Los Angeles has over 18.7 million residents and the San Francisco Bay Area has over 9.6 million residents. Los Angeles is state's most populous city and the nation's second-most populous city. San Francisco is the second-most densely populated major city in the country. Los Angeles County is the country's most populous county, and San Bernardino County is the nation's largest county by area. Sacramento is the state's capital.

# Conversational Context:
# Human - "Hello, how are you?"
# AI - "I'm good, what can I help you with?"
# Human - "What is the captital of California?"
# AI - "Sacramento"
# Human - "Thanks!"
# AI - "You are welcome!"

# ### Response:
# """

# prompt = PromptTemplate(
#     input_variables=["query"],
#     template=template,
# )

# result = pg.Completion.create(
#     model="Nous-Hermes-Llama2-13B",
#     prompt=prompt.format(query="What is the population of LA?")
# )

# print(result['choices'][0]['text'])


In [26]:

# # Let's get the html off of a website.
# fp = urllib.request.urlopen("https://docs.kernel.org/process/submitting-patches.html")
# mybytes = fp.read()
# html = mybytes.decode("utf8")
# fp.close()

# # And convert it to text.
# h = html2text.HTML2Text()
# h.ignore_links = True
# text = h.handle(html)

# print(text)

from PyPDF2 import PdfReader

# Replace 'path_to_your_pdf_file.pdf' with the path to your PDF file
pdf_path = '/content/data/drug_side_effects_summary_cleaned.pdf'
reader = PdfReader(pdf_path)

# Initialize an empty string to accumulate text
text = ''

# Iterate over each page in the PDF
for page in reader.pages:
    # Extract text from the page and append it to the text string
    text += page.extract_text() + "\n"

# Now, `text` contains the text content of the PDF. You can print it or process it further.
print(text[:500])  # Example: print the first 500 characters to understand the structure


# from PyPDF2 import PdfReader

# # Open the PDF file
# pdf_path = '/content/data/drug_side_effects_summary_cleaned.pdf'
# reader = PdfReader(pdf_path)

# # Read each page and extract text
# text = ''
# for page in reader.pages:
#     text += page.extract_text() + "\n"

# # Show the first 500 characters to understand the structure
# text[:500]

Drug Side Effects Summary
Drug and Disease Interactions with Side Effects
1. This particular disease, Acne, interacts with the drug doxycycline and has the following side effects:
- hives, difficult breathing, swelling in your face or throat) or a severe skin reaction (fever, sore
throat, burning in your eyes, skin pain, red or purple skin rash that spreads and causes blistering
and peeling). Seek medical treatment if you have a serious drug reaction that can affect many parts
of your body. Symp


In [31]:
# # Clean things up just a bit.
# text = text.split("### This Page")[1]
# text = text.split("## References")[0]

# # Chunk the text into smaller pieces for injection into LLM prompts.
# text_splitter = CharacterTextSplitter(chunk_size=700, chunk_overlap=50)
# docs = text_splitter.split_text(text)

# # Let's checkout some of the chunks!
# for i in range(0, 3):
#   print("Chunk", str(i+1))
#   print("----------------------------")
#   print(docs[i])
#   print("")

import re

# Function to clean the extracted text
def clean_text(text):
    # Correcting unwanted line breaks and spaces
    text = re.sub(r'-\n', '', text)  # Remove hyphenation
    text = re.sub(r'\n', ' ', text)  # Replace new lines with space
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with single space
    text = text.strip()  # Remove leading and trailing spaces
    return text

# Clean the extracted text
cleaned_text = clean_text(text)

# Return a portion of the cleaned text to verify the cleaning
cleaned_text[:500]

# Define a function to chunk text with specified size and overlap using standard Python
def chunk_text(text, chunk_size=700, overlap=50):
    chunks = []
    start = 0
    while start < len(text):
        # If we're not at the beginning, move back 'overlap' characters for context
        if start > 0:
            start -= overlap
        end = start + chunk_size
        chunks.append(text[start:end])
        start += chunk_size
    return chunks

# Chunk the cleaned text into smaller pieces for LLM input
docs_alternative = chunk_text(cleaned_text, chunk_size=700, overlap=50)

# Prepare to display the first few chunks to verify the result
chunks_to_display_alt = 3
chunks_preview_alt = [docs_alternative[i] for i in range(min(len(docs_alternative), chunks_to_display_alt))]

chunks_preview_alt


# from PyPDF2 import PdfReader
# import re
# from langchain.text_splitter import CharacterTextSplitter

# # Load and clean the PDF text
# pdf_path = '/content/data/drug_side_effects_summary_cleaned.pdf'
# reader = PdfReader(pdf_path)
# text = ''
# for page in reader.pages:
#     text += page.extract_text() + "\n"

# # Basic cleaning function
# def clean_text(text):
#     text = re.sub(r'-\n', '', text)  # Remove hyphenation
#     text = re.sub(r'\n', ' ', text)  # Replace new lines with space
#     text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with single space
#     return text.strip()

# cleaned_text = clean_text(text)


# # Assuming you have specific sections to remove, adjust these lines accordingly
# # cleaned_text = cleaned_text.split("Your Start Marker")[1]
# # cleaned_text = cleaned_text.split("Your End Marker")[0]

# # Chunk the cleaned text
# chunk_size = 700  # Customize based on your LLM's token limit
# chunk_overlap = 50  # Optional overlap to maintain context between chunks
# text_splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
# docs = text_splitter.split_text(cleaned_text)

# # Example to print the first few chunks
# for i, doc in enumerate(docs[:3]):
#     print(f"Chunk {i+1}:")
#     print(doc)
#     print("----------------------------")




# import re

# # Split the text based on a pattern that seems to mark new entries
# # Assuming each entry starts with a numeral followed by ".", as seen in "1. This particular disease..."
# chunks = re.split(r'\n\d+\.', text)

# # Remove any leading or trailing whitespace and unwanted characters from each chunk
# chunks_cleaned = [chunk.strip().replace('\n', ' ').replace('#', '-') for chunk in chunks if chunk.strip()]

# # Show the number of chunks and the first 3 chunks as examples
# len(chunks_cleaned), chunks_cleaned[:3]


['Drug Side Effects Summary Drug and Disease Interactions with Side Effects 1. This particular disease, Acne, interacts with the drug doxycycline and has the following side effects: - hives, difficult breathing, swelling in your face or throat) or a severe skin reaction (fever, sore throat, burning in your eyes, skin pain, red or purple skin rash that spreads and causes blistering and peeling). Seek medical treatment if you have a serious drug reaction that can affect many parts of your body. Symptoms may include: skin rash, fever, swollen glands, flu-like symptoms, muscle aches, severe weakness, unusual bruising, or yellowing of your skin or eyes. This reaction may occur several weeks after y',
 'yes. This reaction may occur several weeks after you began using doxycycline. Doxycycline may cause serious side effects. Call your doctor at once if you have: severe stomach pain, diarrhea that is watery or bloody - throat irritation, trouble swallowing - chest pain, irregular heart rhythm, 

In [33]:
# # Let's take care of some of the formatting so it doesn't conflict with our
# # typical prompt template structure
#  docs = [x.replace('#', '-') for x in docs]


# # Now we need to embed these documents and put them into a "vector store" or
# # "vector db" that we will use for semantic search and retrieval.

# # Embeddings setup
# name="all-MiniLM-L12-v2"
# model = SentenceTransformer(name)

# def embed_batch(batch):
#     return [model.encode(sentence) for sentence in batch]

# def embed(sentence):
#     return model.encode(sentence)

# # LanceDB setup
# os.mkdir(".lancedb")
# uri = ".lancedb"
# db = lancedb.connect(uri)

# # Create a dataframe with the chunk ids and chunks
# metadata = []
# for i in range(len(docs)):
#     metadata.append([
#         i,
#         docs[i]
#     ])
# doc_df = pd.DataFrame(metadata, columns=["chunk", "text"])

# # Embed the documents
# data = with_embeddings(embed_batch, doc_df)

# # Create the DB table and add the records.
# db.create_table("linux", data=data)
# table = db.open_table("linux")
# table.add(data=data)

# Format the chunks to avoid prompt template conflicts
chunks_preview_alt = [x.replace('#', '-') for x in chunks_preview_alt]

# Embeddings setup
name = "all-MiniLM-L12-v2"
model = SentenceTransformer(name)

# Embedding functions
def embed_batch(batch):
    return [model.encode(sentence, show_progress_bar=True) for sentence in batch]

def embed(sentence):
    return model.encode(sentence)

# Ensure the LanceDB directory does not exist already to avoid errors
lancedb_dir = ".lancedb"
if not os.path.exists(lancedb_dir):
    os.mkdir(lancedb_dir)
uri = lancedb_dir
db = lancedb.connect(uri)

# Prepare metadata for embedding
metadata = [[i, chunks_preview_alt] for i, chunks_preview_alt in enumerate(chunks_preview_alt)]
doc_df = pd.DataFrame(metadata, columns=["chunk", "text"])

# Embed the documents
data = with_embeddings(embed_batch, doc_df)

# LanceDB operations
# if not db.has_table("pdf_data"):
db.create_table("pdf_data", data=data)
table = db.open_table("pdf_data")
table.add(data=data)

# Note: Adjust the 'create_table' and 'open_table' to match your dataset/table names


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [37]:
# # Let's try to match a query to one of our documents.
# message = "How many problems should be solved per patch?"
# results = table.search(embed(message)).limit(5).to_df()
# results.head()

message = "What are the side effects of doxycycline for treating Acne?"
results = table.search(embed(message)).limit(5).to_pandas()
print(results.head())


message = "What are the side effects of doxycycline for treating Acne?"
results = table.search(embed(message)).limit(5).to_pandas()
print(results.head())


   chunk                                               text  \
0      0  Drug Side Effects Summary Drug and Disease Int...   
1      0  Drug Side Effects Summary Drug and Disease Int...   
2      2  ea or vomiting, fast heart rate, dark urine, j...   
3      2  ea or vomiting, fast heart rate, dark urine, j...   
4      1  yes. This reaction may occur several weeks aft...   

                                              vector  _distance  
0  [-0.010158041, -0.047626488, 0.008582523, 0.10...   0.404570  
1  [-0.010158041, -0.047626488, 0.008582523, 0.10...   0.404570  
2  [0.039061543, -0.08425787, -0.0030551467, 0.06...   0.530880  
3  [0.039061543, -0.08425787, -0.0030551467, 0.06...   0.530880  
4  [0.031944595, -0.10336819, 0.009887704, 0.0847...   0.871359  
   chunk                                               text  \
0      0  Drug Side Effects Summary Drug and Disease Int...   
1      0  Drug Side Effects Summary Drug and Disease Int...   
2      2  ea or vomiting, fast heart

In [40]:
# # Now let's augment our Q&A prompt with this external knowledge on-the-fly!!!
# template = """### Instruction:
# Read the below input context and respond with a short answer to the given question. Use only the information in the below input to answer the question. If you cannot answer the question, respond with "Sorry, I can't find an answer, but you might try looking in the following resource."

# ### Input:
# Context: {context}

# Question: {question}

# ### Response:
# """
# qa_prompt = PromptTemplate(
#     input_variables=["context", "question"],
#     template=template,
# )

# def rag_answer(message):

#   # Search the for relevant context
#   results = table.search(embed(message)).limit(5).to_df()
#   results.sort_values(by=['_distance'], inplace=True, ascending=True)
#   doc_use = results['text'].values[0]

#   # Augment the prompt with the context
#   prompt = qa_prompt.format(context=doc_use, question=message)

#   # Get a response
#   result = pg.Completion.create(
#       model="Nous-Hermes-Llama2-13B",
#       prompt=prompt
#   )

#   return result['choices'][0]['text']

# response = rag_answer("How many problems should be solved in a single patch?")

# print('')
# print("RESPONSE:", response)



# Assuming the setup for embeddings, LanceDB, and the PromptTemplate are already in place

def rag_answer_drug_side_effects(drug_name):
    # Formulate a question related to drug side effects
    message = f"What are the side effects of {drug_name}?"

    # Search the database for relevant context
    results = table.search(embed(message)).limit(5).to_pandas()  # Adjust based on the correct API call
    results.sort_values(by=['_distance'], inplace=True, ascending=True)
    context = results['text'].iloc[0]  # Use the most relevant document

    # Define the prompt template
    template = """### Instruction:
    Read the below input context and respond with a short answer to the given question. Use only the information in the below input to answer the question. If you cannot answer the question, respond with "Sorry, I can't find an answer, but you might try looking in the following resource."

    ### Input:
    Context: {context}

    Question: {question}

    ### Response:
    """

    # Augment the prompt with the retrieved context
    prompt = template.format(context=context, question=message)

    # Get a response
    result = pg.Completion.create(
    model="Neural-Chat-7B",
    prompt = prompt
)

    # # Here you would call your LLM or any other model to generate an answer based on the prompt
    # # Since we cannot execute dynamic model calls in this environment, we'll simulate a response
    # simulated_response = "Sorry, I can't find an answer, but you might try looking in the following resource."

    return result['choices'][0]['text']

# Example usage
drug_name = "doxycycline"  # Specify the drug of interest
response = rag_answer_drug_side_effects(drug_name)

print("RESPONSE:", response)



RESPONSE: 
Nausea and vomiting, upset stomach, loss of appetite, mild diarrhea, skin rash or itching, darkened skin color, and vaginal itching or discharge.
